In [ ]:
#❶銷售數據彙整 寫入資料庫
import pandas as pd
import sqlite3
import chardet
from datetime import datetime

def read_data_from_file(file_path):
    with open(file_path, "rb") as file:
        content = file.read()
        encoding = chardet.detect(content)["encoding"]

    return pd.read_csv(file_path, encoding=encoding)

def fix_date(date_str):
    if isinstance(date_str, str):            
        dt = datetime.strptime(date_str, '%y-%b')
        dt = dt.replace(day=1)
        return dt.strftime('%Y-%m-%d')
    else:
        return date_str

# 讀取 2021、2022 年和 2023 年的資料
file_path_2122 = "20212022sales.csv"
file_path_2023 = "2023sales.csv"
df_2122 = read_data_from_file(file_path_2122)
df_2023 = read_data_from_file(file_path_2023)

# 修改 2021、2022 年和 2023 年的資料日期
df_2122['交易日期'] = df_2122['交易日期'].apply(fix_date)


# 選擇需要的列並重新命名
df_2122 = df_2122[["交易日期", "成交價格"]].rename(columns={"交易日期": "time", "成交價格": "price"})
df_2023 = df_2023[["交易日期", "價格"]].rename(columns={"交易日期": "time", "價格": "price"})

# 將 "time" 列轉換為 datetime 對象並格式化為 YYYYMMDD
df_2122["time"] = pd.to_datetime(df_2122["time"]).dt.strftime('%Y%m%d')
df_2023["time"] = pd.to_datetime(df_2023["time"]).dt.strftime('%Y%m%d')

# 將 2021、2022 年和修正後的 2023 年的資料合併
df_combined = pd.concat([df_2122, df_2023], ignore_index=True)

# 建立資料庫連接
conn = sqlite3.connect('cb.db')
cursor = conn.cursor()

# 將 DataFrame 存儲到 SQLite 數據庫中的 '2122_sales_date' 表中
df_combined.to_sql('2122_sales_date', conn, if_exists='replace', index=False)

# 關閉資料庫連接
conn.close()

In [ ]:
#❷讀取 2122 年的銷售數據並修復日期格式
import pandas as pd
from datetime import datetime
from pathlib import Path

# 定義文件路徑和列名
FILE_PATH_2122 = "sales_2122.csv"
COL_NAMES_2122 = {"交易日期": "time", "成交價格": "price"}

# 定義函數修復日期格式
def fix_date(date_str):
    if isinstance(date_str, str):
        dt = datetime.strptime(date_str, '%y-%b')
        dt = dt.replace(day=1)
        return dt.strftime('%Y-%m-%d')
    else:
        return date_str


df_2122 = pd.read_csv(FILE_PATH_2122, names=COL_NAMES_2122.keys(), header=0,
                       parse_dates=["交易日期"], date_parser=fix_date)

# 選擇需要的列並重新命名
df_2122 = df_2122[["time", "price"]].rename(columns=COL_NAMES_2122)

# 將 "time" 列轉換為 datetime 對象並格式化為 YYYYMMDD
df_2122["time"] = pd.to_datetime(df_2122["time"]).dt.strftime('%Y%m%d')

# 將數據保存到 CSV 檔案中
OUT_FILE_PATH = "sales_2122.csv"
df_2122.to_csv(OUT_FILE_PATH, index=False, encoding='utf-8-sig')



In [ ]:
#❸網站語言轉換廣告分析報表數據清理
import csv

infn = 'adscost.csv'
outfn = 'adscost_clean.csv'

# 開啟 input.csv 檔案，讀取資料
with open(infn, encoding='utf-8-sig') as csvRFile:
    csvReader = csv.reader(csvRFile)
    listReport = list(csvReader)
    
    # 獲取需要的五個欄位的索引
    header = listReport[0]
    indices = [header.index(col) for col in ['開始','結束時間','成果', '觸及人數', '曝光次數', '每次成果成本', '花費金額 (TWD)']]

# 開啟 output.csv 檔案，寫入資料
with open(outfn, 'w', newline='', encoding='utf-8-sig') as csvOFile:
    csvWriter = csv.writer(csvOFile)

    # Iterate through the rows in listReport and write them to the output file
    for row in listReport:
        # Get the values of the five columns needed
        filtered_row = [row[i] for i in indices]
        csvWriter.writerow(filtered_row)

In [ ]:
# ❹ browser.csv時間格式修正
import csv
from datetime import datetime

infn = 'browser.csv'
outfn = 'browser_clean.csv'

# 開啟 input.csv 檔案，讀取資料
with open(infn, encoding='utf-8-sig') as csvRFile:
    csvReader = csv.reader(csvRFile)
    listReport = list(csvReader)
    
    # 獲取 'Times' 欄位的索引
    header = listReport[0]
    time_index = header.index('Times')

# 開啟 output.csv 檔案，寫入資料
with open(outfn, 'w', newline='', encoding='utf-8-sig') as csvOFile:
    csvWriter = csv.writer(csvOFile)

    # 寫入表頭
    csvWriter.writerow(header)

    # Iterate through the rows in listReport (excluding the header) and write them to the output file
    for row in listReport[1:]:
        # Convert the 'Times' column value to the desired format
        time_value = datetime.strptime(row[time_index], '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d')
        row[time_index] = time_value
        csvWriter.writerow(row)

In [ ]:
#❺ story.csv 資料彙整、篩選/ 編碼轉換
import pandas as pd

# 2022/02-2023/04 storys資料
df1 = pd.read_csv('Jan-11-2023_Apr-11-2023_story.csv')
df2 = pd.read_csv('Jul-11-2022_Oct-10-2022_story.csv')
df3 = pd.read_csv('Apr-11-2022_Jun-10-2022_story.csv')
df4 = pd.read_csv('Feb-01-2022_Apr-10-2022_story.csv')
df5 = pd.read_csv('Jun-11-2022_Jul-10-2022_story.csv')
df6 = pd.read_csv('Oct-11-2022_Jan-10-2023_story.csv')

# 合併
alligstory = pd.concat([df1, df2, df3, df4, df5,df6])

# 另存
selectedigstory = alligstory.loc[:, ['說明', '發佈時間', '永久連結','瀏覽次數','觸及人數',
                          '分享','追蹤人數','個人檔案瀏覽次數','回覆次數','貼圖點按次數','導覽']]

#資料清理
selectedigstory[['發佈日期', '發佈時間']] = selectedigstory['發佈時間'].str.split(' ', expand=True)

from datetime import datetime
selectedigstory['發佈日期'] = selectedigstory['發佈日期'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%Y-%m-%d'))
selectedigstory = selectedigstory.sort_values(by='發佈日期', ascending=False)

selectedigstory.to_csv('story.csv', index=False, encoding='utf-8-sig')